# vite

## hmr

In [ ]:
{
    /*
        ws msg type
    */
    
    let payload = {
        type:null,
        updates:[{
            type:null,
            path:null,
            timestamp:null
        }],
        event:null,
        data:null,
        paths:null,
        err:null,
    }
    switch(payload.type){
        case 'connected':
            setInterval(()=>ws.send('ping'),1000) // 保持 ws keep alive;
            break;
        case 'update':
            if(isFirst && hasError){
                window.location.reload()
            }else{
                clearError();
            }
            
            payload.updates.forEach(item=>{
                if(item.type == 'js-update'){
                    // 如果更新的是js文件
                    queueUpdate(fetchUpdate(item))
                }else{
                    // 如果是css文件
                    link.href = item.path + '?v='+ item.timestamp;
                    // 更新link的 href地址
                }
                
            })
            break;
        case 'custom':
            const cbs = customListenersMap.get(payload.event);
            if(cbs){
                cbs.forEach(cb=>cb(payload.data))
            }
            break;
        case 'full-reload':
            window.location.reload();
            break;
        case 'prune':
            // 清理残留的无用modules 逻辑，消除影响。side effects;
            payload.paths.forEach(path=>{
                const fn = pruneMap.get(path);
                if(fn){
                    fn(dataMap.get(path))
                }
            })
            break;
        case 'error':
            showError();
            break;
        default:
            return payload;
    }

}

## cli 

### cac 命令说明

In [1]:
{
    const cac = require('cac');
    const cli = cac('cus');//显示当前命令说明所指向的package名称
    
    cli.version('1.1.1.1');
    cli.help();
    
    cli.command('build [root]').option('--file file','desc').action((root,options)=>{
        console.log(root,options);
    })
    
    const parsed = cli.parse();
    console.log(JSON.stringify(parsed,null,2));
}

{
  "args": [
    "kernel"
  ],
  "options": {
    "--": [],
    "configPath": "/home/lseyesl/.local/share/jupyter/runtime/kernel-0654e12f-9aef-4132-b62b-e91ee0f8f158.json"
  }
}


### node 执行系统命令

In [3]:
{
    const execa = require('execa');
    const fs = require('fs');
    
    async function main(){
        let res = await execa('ls',['-a']);
        if(res.exitCode === 0) {
            let list = res.stdout.split('\n');
            list.forEach(item=>{
                console.log(item);
            })
        }else{
            console.error(res.stderr);
        }
    }
    
    main();
}

.
..
.ipynb_checkpoints
node_modules
package-lock.json
package.json
reactive.ipynb
vite.ipynb
vue.ipynb
vuex.ipynb


### dev Server

In [ ]:
{
    const http = require('http');
    const ws = require('ws');
    const path = require('path');
    const chokidar = require('chokidar'); //监听文件变化
    const fs = require('fs');
    
    // 文件类型对应
    const type2media = {
        '.html':'text/html',
        '.js':'application/javascript',
        '.css':'text/css',
    }
    
    const server = http.createServer((req,res)=>{
        console.log('req',req.url);
        let queryIdx = req.url.indexOf('?');
        if(queryIdx<0){
            queryIdx = req.url.length;
        }
        
        const _url = req.url.slice(0,queryIdx);
        console.log('_url',_url);
        const _filename = _url == '/'?'index.html':_url.slice(1);
        console.log('_filename',_filename);
        const file = path.resolve(__dirname,'dynamic',_filename);
        
        if(fs.existsSync(file)){
            const ext = path.extname(file);
            console.log('ext:',ext);
            res.writeHead(200,{'Content-Type':type2media[ext]});
            let data = fs.readFileSync(file);
            res.end(data);
        }else{
            res.writeHead(404);
            res.end();
        }
    })
    
    server.listen(8888,err=>{
        if(err) throw err;
        console.log('server listen 8888');
    })
    
    const wss = new ws.Server({
        port:8889
    })
    
    wss.on('connection',socket=>{
        socket.on('message',ms=>{
            console.log('wss-> msg ->:',ms);
        })
        
        chokidar.watch(path.resolve(__dirname,'dynamic')).on('all',(event,_path)=>{
            console.log('file update',event,_path);
            if(event!='change') return ;
            let type = path.extname(_path).slice(1);
            let p = _path.slice(_path.indexOf('dynamic')+7);
            console.log('p1==>',p);
            p = `.${p}`;
            console.log(JSON.stringify({
                type:type,
                path,p
            }))
        })
    })
    
    wss.on('error',error=>{
        console.log('wss error',error);
    })
}

### playwright 测试

In [ ]:
{
    const os = require('os');
    const fs = require('fs-extra');
    const path = require('path');
    const {chromium} = require('playwright-chromium');
    
    const DIR = path.join(__dirname,'wsTemp');
    
    async function main(){
        const browserServer = await chromium.launchServer();
        
        await fs.mkdirp(DIR);
        await fs.writeFile(path.join(DIR,'wsEndpoint'),browserServer.wsEndpoint());
        
        await new Promise(res=>setTimeout(res,2000));
        
        const wsEndpoint = await fs.readFile(path.join(DIR,'wsEndpoint'),'utf-8');
        const browser = await chromium.connect({
            wsEndpoint
        });
        
        const ctx = await browser.newPage();
        browser.close();
        process.exit(0);
    }
}